**Question:** What is ASN1?

**Answser:**
ASN stands for Abstract Syntax Notation.  
ASN1 is a standard IDL for defining data structures that can be serialized and deserialized in a cross-platform way.  
This makes it a suitable configuration file format for oai5g.  

**Sources:**
- https://en.wikipedia.org/wiki/ASN.1

**Question:** What are gNB_ID, nr_cellid and physCellId? Why are they important and how are they used?

**Answser:**
These are unique identifiers that are used in various places in the code base to refer to their respective entities.

- `gNB_ID`:
    - In the RRC mac protocol
    - In the gBN application layer for NGAP messages  

- `nr_cellid`:
    - Used in L2 messages (f1ap)
    - In the RRC mac protocol  

   
- `physCellId`:
    - In the RRC mac protocol

**Sources:**
- oai5g source code

These identifiers are used in the messages of the 5G protocols.

**Question:** How should you pick tracking_area_code? Is this important for our case?

**Answser:**
The Tracking Area is a logical concept of an area where a user can move around without perforing a tracking area update.  
I expect that, given our small coverage region, the tracking are will be the same for the entire region.  
Therefore, the choice of `tracking_area_code` is arbitrary and we can leave it set to 1.  

**Sources:**
- https://www.sharetechnote.com/html/Handbook_LTE_Tracking_Area.html

**Question:** What are mcc and mnc? What should this be if you were Proximus, and what should they be for a test network?

**Answser:**
The `mcc` is the Mobile Country Code and it plays an important role in country identification and roaming functionality.

The `mnc` is the Mobile Network Code and and it is used in combination with the MCC to uniquely identify a mobile network operator (like Proximus).

Proximus specifically has an MCC/MNC tuple of 206/01,  
however for a test network it is adviced to use an MCC where the first digit is 0.  
A good tuple for a test network could be 001/001.

**Sources:**
- https://en.wikipedia.org/wiki/Mobile_country_code
- https://en.wikipedia.org/wiki/Mobile_country_code#Test_networks

**Question:** What value do you pick for dl_carrierBandwidth and why?

**Answser:**
`dl_carrierBandwidth` refers to the downlink carrier bandwidth in terms of the number of resource blocks (PRBs) allocated to the downlink transmission.

Given the 20 MHz of downlink bandwidth (see notebooks),  
the question is: how many resource blocks will fit in?  
A resource block is defined as 12 consecutive subcarriers (in the frequency domain).  
The bandwidth occupied will depend on the sub-carrier spacing (SCS).  
Given a SCS of 15 kHz (changed from the default),  
the following equation must hold:
$$ 20 \mathrm{MHz} = x \mathrm{RB} \times 12  \mathrm{SC/RB} \times 15 \mathrm{kHz/SC} $$
Solving for x gives
$$ x = \frac{20 \times 10^3}{12 \times 15} \approx 111 RB $$

**Sources:**
- ChatGPT May 12 Version

**Question:** What would pdsch_AntennaPorts equal to 4 mean? How would this be used

**Answser:**
The `pdsch` is the Physical Downlink Shared Channel.  
It is a physical channel that delivers user data from the gNB to the UE.  
An important step in the data flow in this channel is the Antenna Port Mapping.  
More antennas improve the spatial diversity and the potential for beamforming.  

**Sources:**
- https://www.sharetechnote.com/html/5G/5G_PDSCH.html#Antenna_Port_Mapping

**Question:** Lookup the formula for EARFCN and calculate the correct values according to your parameter choices 

**Answser:**
`EARFCN` stands for E-UTRA Absolute Radio Frequency Channel Number.  
We are working with band 2 (see previous notebooks),  
which has a downlink frequency of 1960 MHz.  
For FR1, the formula becomes:

$$ N_{REF} = \frac{F_{REF}}{5 \mathrm{kHz}} $$

where $N_{REF}$ is the NR ARFCN and $F_{REF}$ is the NR frequency.  
So, plugging in 1930 MHz we get an ARFCN of 392000.  


**Sources:**
- https://www.sqimway.com/nr_band.php
- https://5g-tools.com/5g-nr-arfcn-calculator/

## Diff of the gnb.conf

```
mcc = 1;  # CHANGED DEFAULT `208`
mnc = 1;  # CHANGED DEFAULT `99`
mnc_length = 3;  # CHANGED DEFAULT `2`
dl_carrierBandwidth                                            = 111;  # CHANGED DEFAULT `106`
dl_subcarrierSpacing                                           = 0;    # CHANGED DEFAULT `1`
ul_subcarrierSpacing                                           = 0;    # CHANGED DEFAULT `1`
```

## Resulting gnb.conf

```
Active_gNBs = ( "gNB-OAI");
# Asn1_verbosity, choice in: none, info, annoying
Asn1_verbosity = "none";

gNBs =
(
 {
    ////////// Identification parameters:
    gNB_ID = 0xe00;

#     cell_type =  "CELL_MACRO_GNB";

    gNB_name  =  "gNB-OAI";

    // Tracking area code, 0x0000 and 0xfffe are reserved values
    tracking_area_code  =  1;

    plmn_list = ({
                  mcc = 1;  # CHANGED DEFAULT `208`
                  mnc = 1;  # CHANGED DEFAULT `99`
                  mnc_length = 3;  # CHANGED DEFAULT `2`
                  snssaiList = (
                                {
                                  sst = 1;
                                  sd  = 0x010203; // 0 false, else true
                                },
                                                                                                                                                                   {
                                  sst = 1;
                                  sd  = 0x000001; // 0 false, else true
                                }
                               );
                 });

    nr_cellid = 12345678L

#     tr_s_preference     = "local_mac"

    ////////// Physical parameters:

    pdsch_AntennaPorts                                        = 1;
    pusch_AntennaPorts                                        = 2;
    min_rxtxtime_pdsch                                        = 5;
    ul_prbblacklist                                           = "51,52,53,54"

    servingCellConfigCommon = (
    {
 #spCellConfigCommon

      physCellId                                                    = 0;

#  downlinkConfigCommon
    #frequencyInfoDL
      # this is 3301.68 MHz + 22*12*30e-3 MHz = 3309.6
      #absoluteFrequencySSB                                          = 620640;
      # this is 3300.60 MHz + 53*12*30e-3 MHz = 3319.68
      absoluteFrequencySSB                                          = 621312;
      # this is 3503.28 MHz + 22*12*30e-3 MHz = 3511.2
      #absoluteFrequencySSB                                          = 634080;
      # this is 3600.48 MHz
      #absoluteFrequencySSB                                          = 640032;
      #dl_frequencyBand                                                 = 78;
      # this is 3301.68 MHz
      #dl_absoluteFrequencyPointA                                       = 620112;
      # this is 3300.60 MHz
      dl_absoluteFrequencyPointA                                       = 620040;
      # this is 3502.56 MHz
      #dl_absoluteFrequencyPointA                                       = 633552;
      # this is 3600.48 MHz
      #dl_absoluteFrequencyPointA                                       = 640032;
      #scs-SpecificCarrierList
        dl_offstToCarrier                                              = 0;
# subcarrierSpacing
# 0=kHz15, 1=kHz30, 2=kHz60, 3=kHz120
        dl_subcarrierSpacing                                           = 0;   # CHANGED DEFAULT `1`
        dl_carrierBandwidth                                            = 111;  # CHANGED DEFAULT `106`
     #initialDownlinkBWP
      #genericParameters
        # this is RBstart=0,L=106 (275*(L-1))+RBstart
        initialDLBWPlocationAndBandwidth                                        = 28875;
# subcarrierSpacing
# 0=kHz15, 1=kHz30, 2=kHz60, 3=kHz120
        initialDLBWPsubcarrierSpacing                                           = 1;
      #pdcch-ConfigCommon
        initialDLBWPcontrolResourceSetZero                                      = 11;
        initialDLBWPsearchSpaceZero                                             = 0;

  #uplinkConfigCommon
     #frequencyInfoUL
      ul_frequencyBand                                                 = 78;
      #scs-SpecificCarrierList
      ul_offstToCarrier                                              = 0;
# subcarrierSpacing
# 0=kHz15, 1=kHz30, 2=kHz60, 3=kHz120
      ul_subcarrierSpacing                                           = 0;   # CHANGED DEFAULT `1`
      ul_carrierBandwidth                                            = 106;
      pMax                                                          = 20;
     #initialUplinkBWP
      #genericParameters
        initialULBWPlocationAndBandwidth                                        = 28875;
# subcarrierSpacing
# 0=kHz15, 1=kHz30, 2=kHz60, 3=kHz120
        initialULBWPsubcarrierSpacing                                           = 1;
      #rach-ConfigCommon
        #rach-ConfigGeneric
          prach_ConfigurationIndex                                  = 98;
#prach_msg1_FDM
#0 = one, 1=two, 2=four, 3=eight
          prach_msg1_FDM                                            = 0;
          prach_msg1_FrequencyStart                                 = 0;
          zeroCorrelationZoneConfig                                 = 12;
          preambleReceivedTargetPower                               = -104;
#preamblTransMax (0...10) = (3,4,5,6,7,8,10,20,50,100,200)
          preambleTransMax                                          = 6;
#powerRampingStep
# 0=dB0,1=dB2,2=dB4,3=dB6
        powerRampingStep                                            = 1;
#ra_ReponseWindow
#1,2,4,8,10,20,40,80
        ra_ResponseWindow                                           = 4;
#ssb_perRACH_OccasionAndCB_PreamblesPerSSB_PR
#1=oneeighth,2=onefourth,3=half,4=one,5=two,6=four,7=eight,8=sixteen
        ssb_perRACH_OccasionAndCB_PreamblesPerSSB_PR                = 3;
#oneHalf (0..15) 4,8,12,16,...60,64
        ssb_perRACH_OccasionAndCB_PreamblesPerSSB                   = 15;
#ra_ContentionResolutionTimer
#(0..7) 8,16,24,32,40,48,56,64
        ra_ContentionResolutionTimer                                = 7;
        rsrp_ThresholdSSB                                           = 19;
#prach-RootSequenceIndex_PR
#1 = 839, 2 = 139
        prach_RootSequenceIndex_PR                                  = 2;
        prach_RootSequenceIndex                                     = 1;
        # SCS for msg1, can only be 15 for 30 kHz < 6 GHz, takes precendence over the one derived from prach-ConfigIndex
        #
        msg1_SubcarrierSpacing                                      = 1,
# restrictedSetConfig
# 0=unrestricted, 1=restricted type A, 2=restricted type B
        restrictedSetConfig                                         = 0,

        msg3_DeltaPreamble                                          = 1;
        p0_NominalWithGrant                                         =-90;

# pucch-ConfigCommon setup :
# pucchGroupHopping
# 0 = neither, 1= group hopping, 2=sequence hopping
        pucchGroupHopping                                           = 0;
        hoppingId                                                   = 40;
        p0_nominal                                                  = -90;
# ssb_PositionsInBurs_BitmapPR
# 1=short, 2=medium, 3=long
      ssb_PositionsInBurst_PR                                       = 2;
      ssb_PositionsInBurst_Bitmap                                   = 1;

# ssb_periodicityServingCell
# 0 = ms5, 1=ms10, 2=ms20, 3=ms40, 4=ms80, 5=ms160, 6=spare2, 7=spare1
      ssb_periodicityServingCell                                    = 2;

# dmrs_TypeA_position
# 0 = pos2, 1 = pos3
      dmrs_TypeA_Position                                           = 0;

# subcarrierSpacing
# 0=kHz15, 1=kHz30, 2=kHz60, 3=kHz120
      subcarrierSpacing                                             = 1;


  #tdd-UL-DL-ConfigurationCommon
# subcarrierSpacing
# 0=kHz15, 1=kHz30, 2=kHz60, 3=kHz120
      referenceSubcarrierSpacing                                    = 1;
      # pattern1
      # dl_UL_TransmissionPeriodicity
      # 0=ms0p5, 1=ms0p625, 2=ms1, 3=ms1p25, 4=ms2, 5=ms2p5, 6=ms5, 7=ms10
      dl_UL_TransmissionPeriodicity                                 = 6;
      nrofDownlinkSlots                                             = 7;
      nrofDownlinkSymbols                                           = 6;
      nrofUplinkSlots                                               = 2;
      nrofUplinkSymbols                                             = 4;

  ssPBCH_BlockPower                                             = -25;
  }

  );


    # ------- SCTP definitions
    SCTP :
    {
        # Number of streams to use in input/output
        SCTP_INSTREAMS  = 2;
        SCTP_OUTSTREAMS = 2;
    };

    ////////// AMF parameters:
        amf_ip_address      = ( { ipv4       = "192.168.70.132";
                                  ipv6       = "192:168:30::17";
                                  active     = "yes";
                                  preference = "ipv4";
                                                                                                                              }
                                                                                                                                          );

        NETWORK_INTERFACES :
        {

           GNB_INTERFACE_NAME_FOR_NG_AMF            = "demo-oai";
           GNB_IPV4_ADDRESS_FOR_NG_AMF              = "192.168.70.129/24";
           GNB_INTERFACE_NAME_FOR_NGU               = "demo-oai";
           GNB_IPV4_ADDRESS_FOR_NGU                 = "192.168.70.129/24";
           GNB_PORT_FOR_S1U                         = 2152; # Spec 2152
        };

  }
);

MACRLCs = (
  {
    num_cc           = 1;
    tr_s_preference  = "local_L1";
    tr_n_preference  = "local_RRC";
#    pusch_TargetSNRx10 = 200;
#    pucch_TargetSNRx10 = 150;
     ulsch_max_frame_inactivity = 0;
  }
);

L1s = (
{
  num_cc = 1;
  tr_n_preference = "local_mac";
  prach_dtx_threshold = 120;
#  pucch0_dtx_threshold = 150;
}
);

RUs = (
    {
       local_rf       = "yes"
         nb_tx          = 2
         nb_rx          = 2
         att_tx         = 0
         att_rx         = 0;
         bands          = [78];
         max_pdschReferenceSignalPower = -27;
         max_rxgain                    = 75;
         eNB_instances  = [0];
         ##beamforming 1x2 matrix: 1 layer x 2 antennas
         bf_weights = [0x00007fff, 0x0000];
         ##beamforming 1x4 matrix: 1 layer x 4 antennas
         #bf_weights = [0x00007fff, 0x0000,0x0000, 0x0000];
         ## beamforming 2x2 matrix:
         # bf_weights = [0x00007fff, 0x00000000, 0x00000000, 0x00007fff];
         ## beamforming 4x4 matrix:
         #bf_weights = [0x00007fff, 0x0000, 0x0000, 0x0000, 0x00000000, 0x00007fff, 0x0000, 0x0000, 0x0000, 0x0000, 0x00007fff, 0x0000, 0x0000, 0x0000, 0x0000, 0x00007fff];

         sf_extension = 0
         sdr_addrs = "addr=192.168.82.52,second_addr=192.168.83.52,clock_source=internal,time_source=internal"
    }
);

THREAD_STRUCT = (
  {
    #three config for level of parallelism "PARALLEL_SINGLE_THREAD", "PARALLEL_RU_L1_SPLIT", or "PARALLEL_RU_L1_TRX_SPLIT"
    parallel_config    = "PARALLEL_RU_L1_SPLIT";
    #two option for worker "WORKER_DISABLE" or "WORKER_ENABLE"
    worker_config      = "WORKER_ENABLE";
  }
);

security = {
  # preferred ciphering algorithms
  # the first one of the list that an UE supports in chosen
  # valid values: nea0, nea1, nea2, nea3
  ciphering_algorithms = ( "nea0" );

  # preferred integrity algorithms
  # the first one of the list that an UE supports in chosen
  # valid values: nia0, nia1, nia2, nia3
  integrity_algorithms = ( "nia2", "nia0" );

  # setting 'drb_ciphering' to "no" disables ciphering for DRBs, no matter
  # what 'ciphering_algorithms' configures; same thing for 'drb_integrity'
  drb_ciphering = "yes";
  drb_integrity = "no";
};

     log_config :
     {
       global_log_level                      ="info";
       global_log_verbosity                  ="medium";
       hw_log_level                          ="info";
       hw_log_verbosity                      ="medium";
       phy_log_level                         ="info";
       phy_log_verbosity                     ="medium";
       mac_log_level                         ="info";
       mac_log_verbosity                     ="high";
       rlc_log_level                         ="info";
       rlc_log_verbosity                     ="medium";
       pdcp_log_level                        ="info";
       pdcp_log_verbosity                    ="medium";
       rrc_log_level                         ="info";
       rrc_log_verbosity                     ="medium";
       f1ap_log_level                         ="debug";
       f1ap_log_verbosity                     ="medium";
    };
```